In [ ]:
# ===============================================================
# BIBLIOTECAS
# ===============================================================
import pycutest as pc # Biblioteca especializada para problemas de otimização de teste
import numpy as np # Computação numérica e operações matemáticas
import time  # Medição de tempo de execução

# ===============================================================
# Método Del Gradiente de Descenso com Armijo (Monótono)
# ===============================================================
def steepest_descent_armijo(p, x0,
                            tau=1e-6,
                            maxit=100000,
                            c1=1e-4,
                            rho=0.5,
                            alpha = 1.0):
    """
    Implementação do Método de Descenso do Gradiente com Busca de Linha de Armijo (Monótono).
    """
    # Ponto inicial
    xk = x0.copy()

    # Avaliação inicial da função e gradiente
    fk, gk = p.obj(xk, gradient=True)
    norm_g = np.linalg.norm(gk)
    hist = [(0, fk, norm_g)]  # Histórico de iterações

    start_time = time.time() # Medição do tempo
    f_evals = 1  # Contador de avaliações da função
    g_evals = 1  # Contador de avaliações do gradiente
    line_searches = 0  # Contador de buscas de linha

    # Loop principal de otimização
    for k in range(1, maxit + 1):
        # Critério de Parada - verifica se o gradiente é suficientemente pequeno
        if norm_g <= tau * (1 + abs(fk)):
            break

        # Busca de Linha de Armijo
        gTdk = -norm_g**2  # Produto interno gradiente × direção
        inner_iter = 0
        max_inner = 20000  # Número máximo de iterações internas

        # Loop interno para ajuste do tamanho do passo
        while inner_iter < max_inner:
            x_new = xk - alpha * gk  # Nova tentativa de ponto
            f_new = p.obj(x_new)  # Avalia função no novo ponto
            f_evals += 1

            # Condição de Armijo - verifica decréscimo suficiente
            if f_new <= fk + c1 * alpha * gTdk:
                line_searches += 1
                break  # Aceita o passo

            alpha *= rho  # Reduz o tamanho do passo
            inner_iter += 1

            # Condição de passo mínimo DENTRO do loop
            if alpha < 1e-18:
                break

        # Atualização das variáveis após busca de linha
        xk = x_new
        fk, gk = p.obj(xk, gradient=True)
        f_evals += 1
        g_evals += 1
        norm_g = np.linalg.norm(gk)
        hist.append((k, fk, norm_g))  # Armazena histórico

        # Condição de passo mínimo FORA do loop (backup)
        if alpha < 1e-18:
            break

        # Condição de máximo de iterações internas alcançado
        if inner_iter >= max_inner:
            break

    elapsed_time = time.time() - start_time
    return xk, fk, hist, elapsed_time, f_evals, g_evals, line_searches

# ===============================================================
# Método Global Barzilai–Borwein (Raydan, 1997)
# ===============================================================
def gbb_raydan(p, x0,
               tau=1e-6,
               epsilon=1e-10,
               y = -1e-5,
               maxit=100000,
               M = 10,
               gamma = 0.2,
               alpha_k  = 1.0):
    """
    Implementação do método Global Barzilai–Borwein (GBB)
    """
    # Ponto Inicial
    xk = x0.copy()

    # Avaliação inicial da função e gradiente
    fk, gk = p.obj(xk, gradient=True)
    fk_list = [fk]  # Lista para critério não monótono
    norm_g = np.linalg.norm(gk)
    hist = [(0, fk, norm_g)] # Histórico de iterações

    start_time = time.time()
    f_evals = 1
    g_evals = 1
    line_searches = 0
    spectral_searches = 0  # Contador de buscas espectrais

    # Loop principal do algoritmo GBB
    for k in range(1, maxit + 1):
        # Critério de Parada
        if norm_g <= tau * (1 + abs(fk)):
            break

        if norm_g == 0:  # Gradiente exatamente zero
            break

        # Limitação de alpha_k para evitar valores extremos
        if alpha_k <= epsilon or alpha_k >= 1/epsilon:
            delta = 1.0 if norm_g > 1.0 else (1.0/norm_g if 1e-5 <= norm_g <= 1.0 else 1e5)
            alpha_k = delta

        lamda = 1/alpha_k  # Parâmetro do passo

        # Critério de decréscimo não monótono - usa histórico de M iterações
        min_k_M = min(k, M)
        f_ref = np.max(fk_list[-min_k_M:])  # Valor de referência

        # Busca de linha - MANTENDO SUA ESTRUTURA ORIGINAL
        r = 0  # Flag de aceitação
        inner_count = 0
        max_inner = 20000  # Limite de segurança

        # Loop interno de busca de linha
        while r != 1 and inner_count < max_inner:
            x_new = xk - lamda * gk
            f_new = p.obj(x_new)
            f_evals += 1

            # Condição de aceitação não monótona
            if f_new <= f_ref + y * lamda * norm_g**2:
                # Aceitar o passo
                lamda_k = lamda
                g_new = p.grad(x_new)
                g_evals += 1
                y_km1 = g_new - gk  # Diferença de gradientes
                spectral_searches += 1  # CONTADOR DE BUSCAS ESPECTRAIS

                # Cálculo de alpha_k (GBB) - ESTO É UMA BUSCA ESPECTRAL
                if np.dot(gk, gk) > 1e-12:  # Evita divisão por zero
                    alpha_k = -(np.dot(gk, y_km1)) / (lamda_k * np.dot(gk, gk))

                # Atualização das variáveis
                xk = x_new
                fk = f_new
                gk = g_new
                norm_g = np.linalg.norm(gk)
                fk_list.append(fk)
                r = 1  # Sai do loop

            else:
                # Redução de passo - passo rejeitado
                lamda *= gamma
                alpha_k = 1.0 / lamda
                line_searches += 1
                r = 0  # Permanece no loop

            inner_count += 1

            # CONDIÇÃO CRÍTICA - DENTRO do loop interno
            if alpha_k < 1e-18:  # Passo muito pequeno
                break

        # Verificações após o loop interno
        if inner_count >= max_inner:  # Muitas iterações internas
            break

        if lamda < 1e-18:  # Passo insignificante
            break

        hist.append((k, fk, norm_g))  # Atualiza histórico

    elapsed_time = time.time() - start_time
    return xk, fk, hist, elapsed_time, f_evals, g_evals, line_searches, spectral_searches

# ===============================================================
# Bloco principal de execução para PyCUTEst
# ===============================================================
def run_cutest_optimization(problema_nombre):
    """
    Executa a otimização usando a dimensão padrão do problema
    """
    print("\n" + "#"*60)
    print(f"## AVALIANDO PROBLEMA: {problema_nombre}")
    print("#"*60)

    try:
        # SÓ usa import_problem - sem especificar dimensão
        p = pc.import_problem(problema_nombre)
        print(f"Problema carregado com dimensão padrão")

    except Exception as e:
        print(f"Erro ao carregar o problema {problema_nombre}: {e}")
        return None

    # Informação do problema
    x0 = p.x0  # Ponto inicial
    dimension = p.n  # Dimensão do problema
    f0, g0 = p.obj(x0, gradient=True)  # Avaliação inicial

    print(f"Função: {p.name}")
    print(f"Dimensão do problema: {dimension}")

    # Armazenar resultados para análise
    results = []

    # ------------------- Descenso Metodo do gradiente -------------------
    try:
        # Executa método Armijo
        x_opt_arm, f_opt_arm, hist_arm, time_arm, f_evals_arm, g_evals_arm, ls_arm = steepest_descent_armijo(p, x0.copy())

        final_iter_arm = len(hist_arm) - 1
        final_norm_g_arm = hist_arm[-1][2] if hist_arm else float('inf')

        # Determinar estado de convergência
        if final_norm_g_arm <= 1e-6 * (1 + abs(f_opt_arm)):
            convergence_arm = "CONVERGIU"
        elif final_iter_arm >= 10000:
            convergence_arm = "EXCEDEU ITERAÇÕES MÁXIMAS"
        elif hist_arm and hist_arm[-1][2] > 1e-3:
            convergence_arm = "GRADIENTE NÃO PRÓXIMO DE ZERO"
        else:
            convergence_arm = "NÃO CONVERGIU"

        armijo_result = {
            'method': 'Metodo do gradiente',
            'iterations': final_iter_arm,
            'time': time_arm,
            'final_norm_g': final_norm_g_arm,
            'final_f': f_opt_arm,
            'convergence': convergence_arm,
            'f_evals': f_evals_arm,
            'g_evals': g_evals_arm,
            'line_searches': ls_arm,  # BUSCAS LINEARES PARA METODO DO GRADIENTE
            'spectral_searches': 0,   # ARMIJO NÃO USA BUSCAS ESPECTRAIS
            'history': hist_arm
        }
        results.append(armijo_result)

    except Exception as e:
        print(f"    Erro em Armijo: {e}")

    # ------------------- GBB Raydan -------------------
    try:
        # Executa método GBB
        x_opt_gbb, f_opt_gbb, hist_gbb, time_gbb, f_evals_gbb, g_evals_gbb, ls_gbb, ss_gbb = gbb_raydan(p, x0.copy())

        final_iter_gbb = len(hist_gbb) - 1
        final_norm_g_gbb = hist_gbb[-1][2] if hist_gbb else float('inf')

        # Determinar estado de convergência
        if final_norm_g_gbb <= 1e-6 * (1 + abs(f_opt_gbb)):
            convergence_gbb = "CONVERGIU"
        elif final_iter_gbb >= 10000:
            convergence_gbb = "EXCEDEU ITERAÇÕES MÁXIMAS"
        elif hist_gbb and hist_gbb[-1][2] > 1e-3:
            convergence_gbb = "GRADIENTE NÃO PRÓXIMO DE ZERO"
        else:
            convergence_gbb = "NÃO CONVERGIU"

        gbb_result = {
            'method': 'GBB',
            'iterations': final_iter_gbb,
            'time': time_gbb,
            'final_norm_g': final_norm_g_gbb,
            'final_f': f_opt_gbb,
            'convergence': convergence_gbb,
            'f_evals': f_evals_gbb,
            'g_evals': g_evals_gbb,
            'line_searches': ls_gbb,
            'spectral_searches': ss_gbb,  # BUSCAS ESPECTRAIS PARA GBB
            'history': hist_gbb
        }
        results.append(gbb_result)

    except Exception as e:
        print(f"   Erro em GBB: {e}")

    # ===============================================================
    # ANÁLISE COMPARATIVA
    # ===============================================================
    print("\n" + "="*70)
    print("ANÁLISE COMPARATIVA")
    print("="*70)

    if results:
        # Melhor modelo em iterações
        best_iter = min(results, key=lambda x: x['iterations'])
        print(f"\n🏆 MELHOR MODELO EM ITERAÇÕES: {best_iter['method']}")
        print(f"   Iterações: {best_iter['iterations']}")
        print(f"   Tempo: {best_iter['time']:.4f} segundos")
        print(f"   Norma do gradiente: {best_iter['final_norm_g']:.3e}")
        if best_iter['method'] == 'Armijo':
            print(f"   Buscas lineares: {best_iter['line_searches']}")  # PARA METODO DO GRADIENTE
        else:
            print(f"   Buscas espectrais: {best_iter['spectral_searches']}")  # PARA GBB
        print(f"   Estado: {best_iter['convergence']}")

        # Melhor modelo em tempo
        best_time = min(results, key=lambda x: x['time'])
        print(f"\n⚡ MELHOR MODELO EM TEMPO: {best_time['method']}")
        print(f"   Tempo: {best_time['time']:.4f} segundos")
        print(f"   Iterações: {best_time['iterations']}")
        print(f"   Norma do gradiente: {best_time['final_norm_g']:.3e}")
        if best_time['method'] == 'Armijo':
            print(f"   Buscas lineares: {best_time['line_searches']}")  # PARA METODO DO GRADIENTE
        else:
            print(f"   Buscas espectrais: {best_time['spectral_searches']}")  # PARA GBB
        print(f"   Estado: {best_time['convergence']}")

        # Melhor modelo em gradiente (mais próximo de zero)
        best_grad = min(results, key=lambda x: x['final_norm_g'])
        print(f"\n🎯 MELHOR MODELO EM GRADIENTE: {best_grad['method']}")
        print(f"   Norma do gradiente: {best_grad['final_norm_g']:.3e}")
        print(f"   Iterações: {best_grad['iterations']}")
        print(f"   Tempo: {best_grad['time']:.4f} segundos")
        if best_grad['method'] == 'Armijo':
            print(f"   Buscas lineares: {best_grad['line_searches']}")  # PARA METODO DO GRADIENTE
        else:
            print(f"   Buscas espectrais: {best_grad['spectral_searches']}")  # PARA GBB
        print(f"   Estado: {best_grad['convergence']}")

    # ===============================================================
    # RESUMO FINAL
    # ===============================================================
    print("\n" + "="*70)
    print("RESUMO FINAL")
    print("="*70)
    print(f"Função avaliada: {p.name}")
    print(f"Dimensão do problema: {dimension}")
    print(f"Valor inicial f(x0): {f0:.6e}")
    print(f"Norma inicial do gradiente: {np.linalg.norm(g0):.3e}")

    if results:
        print(f"\nAlgoritmos avaliados: {len(results)}")
        for result in results:
            print(f"\n{result['method']}:")
            print(f"  • Iterações: {result['iterations']}")
            print(f"  • Tempo: {result['time']:.4f}s")
            print(f"  • Norma gradiente: {result['final_norm_g']:.3e}")
            if result['method'] == 'Armijo':
                print(f"  • Buscas lineares: {result['line_searches']}")  # PARA METODO DO GRADIENTE
            else:
                print(f"  • Buscas espectrais: {result['spectral_searches']}")  # PARA GBB
            print(f"  • Estado: {result['convergence']}")
            print(f"  • Avaliações de f: {result['f_evals']}")
            print(f"  • Avaliações de g: {result['g_evals']}")

    return results

if __name__ == "__main__":
    # Lista de problemas disponíveis para teste:
    #"ARGLINA", "ARGLINB", "BA-L1SPLS", "BIGGS6", "BROWNAL", "COATING",
    #"FLETCHCR", "GAUSS2LS", "GENROSE", "HAHN1LS", "HEART6LS", "HILBERTB",
    #"HYDCAR6LS", "LANCZOS1LS", "LANCZOS2LS", "LRIJCNN1", "LUKSAN12LS",
    #"LUKSAN16LS", "OSBORNEA", "PALMER1C", "PALMER3C", "PENALTY2", "PENALTY3",
    #"QING", "ROSENBR", "STRTCHDV", "TESTQUAD", "THURBERLS", "TRIGON1",
    #"TOINTGOR
    resultados = run_cutest_optimization("ROSENBR")